In [1]:
library(Matching)
library(tidyverse)

Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::select() masks MASS::select()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_lactate_flag,lab_lactate_first,lab_lactate_min,lab_lactate_max,lab_lactate_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,0,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1.4,1.4,1.4,0,0,1,0,0.5444486,1.836721
278148,169700,49395,0,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2569337,1.345775
274174,103722,14855,0,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1.6,1.6,2.6,1,0,1,0,0.3264142,3.063592
214954,170515,54642,0,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,3.8,2.2,3.8,1,1,1,0,0.8063102,1.240217
222457,183493,96815,0,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.2239520,1.288580
264413,134244,81436,0,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,1,1.4,1.4,1.4,0,0,1,0,0.3457691,2.892104


In [4]:
set.seed(4958)

In [5]:
ps_matches <- Match(Y = full_data$mort_28_day_int, Tr = full_data$echo_int,
                    X = full_data$ps, M = 1, estimand = "ATT", caliper = 0.01,
                    exact = FALSE, replace = FALSE, ties = FALSE)

Warning message in Match(Y = full_data$mort_28_day_int, Tr = full_data$echo_int, :
“replace==FALSE, but there are more (weighted) treated obs than control obs.  Some treated obs will not be matched.  You may want to estimate ATC instead.”

In [ ]:
saveRDS(ps_matches, file = file.path(data_dir, "ps_matches.rds"))

In [6]:
summary(ps_matches)


Estimate...  -0.063843 
SE.........  0.015715 
T-stat.....  -4.0625 
p.val......  4.855e-05 

Original number of observations..............  6361 
Original number of treated obs...............  3262 
Matched number of observations...............  1629 
Matched number of observations  (unweighted).  1629 

Number of obs dropped by 'exact' or 'caliper'  1633 



In [7]:
outcome <- data.frame(echo_pt=full_data$mort_28_day_int[ps_matches$index.treated],
                      match_pt=full_data$mort_28_day_int[ps_matches$index.control])

In [8]:
head(outcome)

echo_pt,match_pt
0,0
0,0
0,0
0,0
1,0
0,1


In [9]:
tab <- table(outcome$echo_pt, outcome$match_pt, dnn = c("Echo","Non-Echo"))
tab
tab[2, 1] / tab[1, 2]
paste("95% Confint",
      round(exp(c(log(tab[2, 1] / tab[1, 2]) - qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]),
                  log(tab[2, 1] / tab[1, 2]) + qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]))), 7))
mcnemar.test(tab) # for 1-1 pairs

    Non-Echo
Echo   0   1
   0 846 383
   1 279 121

[1] 0.7284595

[1] "95% Confint 0.6243207" "95% Confint 0.8499691"


	McNemar's Chi-squared test with continuity correction

data:  tab
McNemar's chi-squared = 16.026, df = 1, p-value = 6.249e-05


In [10]:
(tab[2, 2] + tab[1, 2]) / sum(tab)
(tab[2, 2] + tab[2, 1]) / sum(tab)

[1] 0.3093923

[1] 0.2455494

In [11]:
mortality <- data.frame(echo = c(0, 1),
                        mortality = c((tab[2, 2] + tab[1, 2]) / sum(tab),
                                      (tab[2, 2] + tab[2, 1]) / sum(tab)))
mortality

echo,mortality
0,0.3093923
1,0.2455494


In [12]:
mort_fmt <- mortality %>%
mutate(echo = case_when(
    echo == 0 ~ "Non_echo",
    echo == 1 ~ "Echo"
)) %>%
select(echo, mortality) %>%
spread(echo, mortality) %>%
mutate(Method = "PS Match") %>%
select(Method, Echo, Non_echo)
mort_fmt

Method,Echo,Non_echo
PS Match,0.2455494,0.3093923


In [13]:
data.table::fwrite(mort_fmt, file.path(data_dir, "mort_ps.csv"))